# Genral Clean 

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

In [2]:
# read th data 
df = pd.read_csv('K_dirty_data.csv')

# which columns we have
df.columns

Index(['Timestamp', 'Name', 'Location', 'Age', 'Gender', 'Interested_in',
       'Current_status', 'Usually', 'looking_for', 'Confidence', 'Bars',
       'Social_network', 'Dating apps', 'Through_a_friend', 'Street',
       'Dating_apps_you_use', 'use_apps_for', 'experience', 'frequently_use',
       'paid', 'swipes_are', 'Reasons_you_dont_like', 'Kukumbo?'],
      dtype='object')

In [3]:
# drop not relenant columns
cols_to_drop = ['Timestamp', 'Name']
df = df.drop(columns=cols_to_drop)

In [4]:
# check if there is Nan values 
df.isnull().sum()

Location                 0
Age                      0
Gender                   0
Interested_in            0
Current_status           0
Usually                  0
looking_for              0
Confidence               0
Bars                     0
Social_network           0
Dating apps              0
Through_a_friend         0
Street                   0
Dating_apps_you_use      0
use_apps_for             0
experience               0
frequently_use           0
paid                     0
swipes_are               0
Reasons_you_dont_like    0
Kukumbo?                 0
dtype: int64

In [5]:
data_dict = {
    # Current_status AND Usually
    'In a relationship': 'In_relationship',
    'In a Relationship': 'In_relationship',
    'In an open relationship': 'Open_relationship',
    'In an Open Relationship': 'Open_relationship',
    
    #Interseted_in 
    'Men;Women':'Both',
    'Men;Women;Other':'Both',
    
    #Location
    # not-sure that is the right way to do it. 
    'Jerusalm and around': 'Jerusalem_and_around',
    'I do not live in Israel': 'Not_live_in_Israel',
    
    #Age
    '18-22': 1,
    '23-27': 2,
    '28-32': 3,
    '32-36': 4,
    '36+': 5,
    
    #Confidence
    'I\'m always too shy to do it': 0,
    'I am shy, most of the time I don\'t do it': 1,
    'I am shy, but sometimes I do it anyway': 2,
    'I have a lot of self confidence': 3,
    
    #Dating_apps_you_use
    'I didn\'t use any of the following ': 'None',
    
    #paid
    'Never': 0,
    'Tried it once or twice': 1,
    'Sure, several times':2,
    
    #swipes_are
    'Cheap, feels a bit wierd to swipe people\'s pictures so fast': 'Cheap',
    'Fun, it is nice to look and "judge" people quickly': 'Fun',
    
    #kukumbo?
    'Yes, sounds cool': 'Yes',
    'No, not for me': 'No'
     
}

In [6]:
#replace the data-frame valutes according to the dictionary
df = df.replace(list(data_dict.keys()), list(data_dict.values()))

In [7]:
# set the labels 
labels = df['Kukumbo?']

# Split features 

 Split look_for to 2 columns:
 * look_for_Long_term = 1 if looking for long term relationship, otherwise 0
 * look_for_Short_term = 1 if looking for short term relationship, otherwise 0
 * Both = assign 1 in look_for_Long_term and look_for_Short_term columns

In [8]:
data_look_for = pd.get_dummies(df['looking_for'])
data_look_for.rename(columns={'Long-term Dating\ a Relationship': 'look_for_Long_term', 'Hookups \ Short-term Dating': 'look_for_Short_term'}, inplace=True)
data_look_for['look_for_Long_term'][data_look_for['Both'] == 1] = 1
data_look_for['look_for_Short_term'][data_look_for['Both'] == 1] = 1
data_look_for = data_look_for.drop(columns=['None', 'Both'])

Split Dating_apps_you_use column. 
* Create columns for each daing app
* 1 if the instance use the app, otherwise 0.

Split use_apps_for column
* Create column for each goal for using
* if assgin more than one goal then both column get 1
* didn't use gives 0 to all of the columns

Split Reasons_you_dont_like column
* Create column for each reason 
* assign 1 for chosen reason , otherwise 0 
* since we have few comment of other. Most of them not informative we will ignore them. 

In [9]:
def update_newCols_for_instance(instance, columns, data_lst, return_counts=False):
    count = 0
    for i, col in enumerate(columns):
        if data_lst[i] in instance:
            col.append(1)
            count += 1
        else:
            col.append(0)
    
    if return_counts:
        return count

In [10]:
def dataFrame_From_columns(names_lst, cols_lst):
    data = ()
    for names, cols in zip(names_lst, cols_lst):
        data += (pd.DataFrame({name : col for name, col in zip(names, cols)}),)
    return data

In [11]:
# prepare data for split Dating_apps_you_use column
data_DA = df['Dating_apps_you_use'].str.split(';')
dateApps_lst = ['Tinder', 'Bumble', 'OkCupid', 'Grinder', 'Hinge','Atraf', 'Badoo']
columns = [[] for i in range(len(dateApps_lst))]
num_of_apps = []

# prepare data for split use_apps_for column
data_use_for = df['use_apps_for'].str.split(';')
use_for_ans = ['Hookups \\ Short-term Dating', 'Long-term Dating \ Relationship', 'Friends']
use_for_names = ['use_for_Long_term', 'use_for_Short_term', 'use_for_Friends']
use_for_cols = [[] for i in range(len(use_for_ans))]


# prepare data for split Reasons_you_dont_like column
data_reasons = df['Reasons_you_dont_like'].str.split(';')
reasons_lst = ['I\'m affraid of bad people', 'Don\'t want to be seen there', 'Too much not my type people there', 'Couldn\'t find somebody I liked enough']
reasons_names = ['affraid_bad_people', 'not_seen_there', 'too_much_not_myType', 'cannot_find_somebody_I_like']
reasons_cols = [[] for i in range(len(reasons_lst))]

for x_DA, x_use_for, x_reasons in zip(data_DA, data_use_for, data_reasons) : 
    # delete white spaces
    x_DA = [s.replace(" ", "") for s in x_DA]
    #delete ""
    x_reasons = [s.replace("\"", "") for s in x_reasons]
    
    #split Dating_apps_you_use
    count_apps = update_newCols_for_instance(x_DA, columns, dateApps_lst, return_counts=True)
    num_of_apps.append(count_apps)
    
    #split of use_apps_for 
    update_newCols_for_instance(x_use_for, use_for_cols, use_for_ans, return_counts=False)
    
    #split of Reasons_you_dont_like
    update_newCols_for_instance(x_reasons, reasons_cols, reasons_lst, return_counts=False)
    
Apps, numApps, use_for, reasons = dataFrame_From_columns([dateApps_lst,['num_apps'], use_for_names, reasons_names], [columns, [num_of_apps], use_for_cols, reasons_cols])
#numApps = pd.DataFrame({'num_apps': num_of_apps})

In [12]:
#drop the non_relevant columns 
df.drop(columns =['looking_for', 'Dating_apps_you_use', 'use_apps_for', 'Reasons_you_dont_like', 'Kukumbo?'], inplace=True)

#concat the clean data from the columns we droped
df = pd.concat([df, data_look_for, Apps, numApps, use_for, reasons, labels], axis=1)

In [13]:
df

Location  Age  Gender Interested_in   Current_status          Usually  \
0     Center    2  Female           Men  In_relationship  In_relationship   
1     Center    2    Male         Women  In_relationship           Single   
2     Center    2    Male         Women  In_relationship  In_relationship   
3      North    2    Male         Women           Single           Single   
4     Center    2  Female           Men           Single           Single   
..       ...  ...     ...           ...              ...              ...   
407   Center    2    Male         Women           Single           Single   
408    North    2  Female           Men           Single           Single   
409   Center    2  Female           Men           Single           Single   
410   Center    2  Female          Both  In_relationship  In_relationship   
411   Center    2    Male         Women           Single           Single   

     Confidence  Bars  Social_network  Dating apps  ...  Badoo  num_apps  \
0             2     3               3            3  ...      0         0   
1             1     3               1            2  ...      0         2   
2             1     2               1            3  ...      0         1   
3             1     3               4            1  ...      0         1   
4             2     2               3            5  ...      0         2   
..          ...   ...             ...          ...  ...    ...       ...   
407           2     3               2            3  ...      0         3   
408           3     3               4            4  ...      0         0   
409           1     4               3            1  ...      0         1   
410           3     4               2            2  ...      0         0   
411           0     4               3            2  ...      0         2   

     use_for_Long_term  use_for_Short_term  use_for_Friends  \
0                    0                   0                0   
1                    1                   1                0   
2                    1                   0                0   
3                    0                   0                0   
4                    0                   1                0   
..                 ...                 ...              ...   
407                  1                   0                0   
408                  0                   0                0   
409                  0                   1                0   
410                  0                   0                0   
411                  1                   1                0   

    affraid_bad_people  not_seen_there  too_much_not_myType  \
0                    0               1                    1   
1                    0               0                    0   
2                    0               1                    1   
3                    0               1                    0   
4                    1               0                    1   
..                 ...             ...                  ...   
407                  0               0                    0   
408                  1               1                    0   
409                  0               0                    0   
410                  1               1                    0   
411                  0               1                    0   

     cannot_find_somebody_I_like  Kukumbo?  
0                              0       Yes  
1                              1       Yes  
2                              1        No  
3                              0       Yes  
4                              1       Yes  
..                           ...       ...  
407                            0       Yes  
408                            0       Yes  
409                            1       Yes  
410                            0        No  
411                            1       Yes  

[412 rows x 34 columns]

In [14]:
#statistic on the numeric values
df.describe()

Age  Confidence        Bars  Social_network  Dating apps  \
count  412.000000  412.000000  412.000000      412.000000   412.000000   
mean     1.849515    2.014563    2.888350        2.516990     2.424757   
std      0.636807    0.834057    1.211125        1.192924     1.322340   
min      1.000000    0.000000    1.000000        1.000000     1.000000   
25%      1.000000    1.000000    2.000000        1.000000     1.000000   
50%      2.000000    2.000000    3.000000        2.000000     2.000000   
75%      2.000000    3.000000    4.000000        3.000000     3.000000   
max      5.000000    3.000000    5.000000        5.000000     5.000000   

       Through_a_friend      Street  experience  frequently_use        paid  \
count        412.000000  412.000000  412.000000      412.000000  412.000000   
mean           3.735437    2.599515    1.876214        1.881068    0.245146   
std            1.173680    1.304046    1.868347        2.010418    0.536367   
min            1.000000    1.000000    0.000000        0.000000    0.000000   
25%            3.000000    1.000000    0.000000        0.000000    0.000000   
50%            4.000000    3.000000    2.000000        1.000000    0.000000   
75%            5.000000    4.000000    3.000000        3.000000    0.000000   
max            5.000000    5.000000    6.000000        6.000000    2.000000   

       ...       Atraf       Badoo    num_apps  use_for_Long_term  \
count  ...  412.000000  412.000000  412.000000         412.000000   
mean   ...    0.024272    0.024272    1.194175           0.402913   
std    ...    0.154079    0.154079    1.298011           0.491080   
min    ...    0.000000    0.000000    0.000000           0.000000   
25%    ...    0.000000    0.000000    0.000000           0.000000   
50%    ...    0.000000    0.000000    1.000000           0.000000   
75%    ...    0.000000    0.000000    2.000000           1.000000   
max    ...    1.000000    1.000000    5.000000           1.000000   

       use_for_Short_term  use_for_Friends  affraid_bad_people  \
count          412.000000       412.000000          412.000000   
mean             0.402913         0.065534            0.218447   
std              0.491080         0.247767            0.413694   
min              0.000000         0.000000            0.000000   
25%              0.000000         0.000000            0.000000   
50%              0.000000         0.000000            0.000000   
75%              1.000000         0.000000            0.000000   
max              1.000000         1.000000            1.000000   

       not_seen_there  too_much_not_myType  cannot_find_somebody_I_like  
count      412.000000           412.000000                   412.000000  
mean         0.334951             0.415049                     0.305825  
std          0.472547             0.493329                     0.461316  
min          0.000000             0.000000                     0.000000  
25%          0.000000             0.000000                     0.000000  
50%          0.000000             0.000000                     0.000000  
75%          1.000000             1.000000                     1.000000  
max          1.000000             1.000000                     1.000000  

[8 rows x 27 columns]

In [15]:
df.shape

(412, 34)

In [16]:
#check the data cololumns and values 

cols = df.columns
for col in cols:
    print(col)
    print(np.unique(df[col], return_counts=True))
    print('---')

Location
(array(['Center', 'Jerusalem_and_around', 'North', 'Not_live_in_Israel',
       'South'], dtype=object), array([337,   9,  35,   9,  22], dtype=int64))
---
Age
(array([1, 2, 3, 4, 5], dtype=int64), array([104, 278,  21,   6,   3], dtype=int64))
---
Gender
(array(['Female', 'Male'], dtype=object), array([202, 210], dtype=int64))
---
Interested_in
(array(['Both', 'Men', 'Other', 'Women'], dtype=object), array([ 19, 197,   2, 194], dtype=int64))
---
Current_status
(array(['In_relationship', 'Open_relationship', 'Single'], dtype=object), array([157,   8, 247], dtype=int64))
---
Usually
(array(['In_relationship', 'Open_relationship', 'Single'], dtype=object), array([157,   2, 253], dtype=int64))
---
Confidence
(array([0, 1, 2, 3], dtype=int64), array([ 17,  89, 177, 129], dtype=int64))
---
Bars
(array([1, 2, 3, 4, 5], dtype=int64), array([ 65,  94, 113, 102,  38], dtype=int64))
---
Social_network
(array([1, 2, 3, 4, 5], dtype=int64), array([104, 103, 118,  62,  25], dtype=int64))
-

# Change all to numbers

In [17]:
second__dict = {
    
    # ** Categorial features ** #
    #Location
    'Center': 0,
    'Jerusalem_and_around': 1,
    'North': 2,
    'South': 3,
    'Not_live_in_Israel': 4,
    
    
    #Gender
    'Female': 0,
    'Male': 1, 
    
    # InterstedIn
    'Both': 3,
    'Men':2,
    'Women':1,
    'Other':0,
    
    # Current_status AND Usually
    'In_relationship': 0,
    'Open_relationship': 1,
    'Single':2,
    
    # Swipes_are
    'Cheap': 0,
    'Fun': 1,
    
    #Kukumbo?
    'No': 0,
    'Yes': 1
    
    
}

In [18]:
#replace the data-frame valutes according to the dictionary
df = df.replace(list(second__dict.keys()), list(second__dict.values()))

In [20]:
#save to csv 
df.to_csv('K_clean_data.csv', index=False)

In [21]:
df

Location  Age  Gender  Interested_in  Current_status  Usually  \
0           0    2       0              2               0        0   
1           0    2       1              1               0        2   
2           0    2       1              1               0        0   
3           2    2       1              1               2        2   
4           0    2       0              2               2        2   
..        ...  ...     ...            ...             ...      ...   
407         0    2       1              1               2        2   
408         2    2       0              2               2        2   
409         0    2       0              2               2        2   
410         0    2       0              3               0        0   
411         0    2       1              1               2        2   

     Confidence  Bars  Social_network  Dating apps  ...  Badoo  num_apps  \
0             2     3               3            3  ...      0         0   
1             1     3               1            2  ...      0         2   
2             1     2               1            3  ...      0         1   
3             1     3               4            1  ...      0         1   
4             2     2               3            5  ...      0         2   
..          ...   ...             ...          ...  ...    ...       ...   
407           2     3               2            3  ...      0         3   
408           3     3               4            4  ...      0         0   
409           1     4               3            1  ...      0         1   
410           3     4               2            2  ...      0         0   
411           0     4               3            2  ...      0         2   

     use_for_Long_term  use_for_Short_term  use_for_Friends  \
0                    0                   0                0   
1                    1                   1                0   
2                    1                   0                0   
3                    0                   0                0   
4                    0                   1                0   
..                 ...                 ...              ...   
407                  1                   0                0   
408                  0                   0                0   
409                  0                   1                0   
410                  0                   0                0   
411                  1                   1                0   

     affraid_bad_people  not_seen_there  too_much_not_myType  \
0                     0               1                    1   
1                     0               0                    0   
2                     0               1                    1   
3                     0               1                    0   
4                     1               0                    1   
..                  ...             ...                  ...   
407                   0               0                    0   
408                   1               1                    0   
409                   0               0                    0   
410                   1               1                    0   
411                   0               1                    0   

     cannot_find_somebody_I_like  Kukumbo?  
0                              0         1  
1                              1         1  
2                              1         0  
3                              0         1  
4                              1         1  
..                           ...       ...  
407                            0         1  
408                            0         1  
409                            1         1  
410                            0         0  
411                            1         1  

[412 rows x 34 columns]